In [1]:
import os
import sys
sys.path.insert(0, "%s/hgcalEnv/lib/python3.6/site-packages/"%os.getcwd())

import uproot4
import awkward as ak
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ROOT

Welcome to JupyROOT 6.18/00


In [2]:
fName = "root://cmseos.fnal.gov//store/user/rverma/Output/cms-hgcal-econd/ntuple/ntuple_Events_4807434_0.root"
pd.options.mode.chained_assignment = None
adcLSB_ = 100./1024.
tdcLSB_ = 10000./4096.
tdcOnsetfC_ = 60.

jobNumber_eventOffset = 100000
negZ_eventOffset      =  50000
cassette_eventOffset  =   1000

linkMap = pd.read_csv('geomInfo/eLinkInputMapFull.csv')
calibrationCells = pd.read_csv('geomInfo/calibrationCells.csv')

waferRemap = pd.read_csv('geomInfo/WaferNumberingMatch.csv')[['layer','waferu','waferv','C1_waferu','C1_waferv','Cassette']]
#waferRemap.set_index(['layer','waferu','waferv'],inplace=True)

In [3]:
#pd.set_option('display.max_rows', None)
#linkMap
#calibrationCells
#waferRemap

In [4]:
def getTree(fName): 
    treeName = 'hgcalTriggerNtuplizer/HGCalTriggerNtuple'
    print ("File %s"%fName)
    try:
        _tree = uproot4.open(fName,xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))[treeName]
        return _tree
    except:
        print ("---Unable to open file, skipping")
        return None

In [5]:
def processDF(fulldf, outputName="test.csv", check=True):
    #if data is ADC, charge = data * adcLSB
    #else data is TDC, charge = tdcStart  + data*tdcLSB
    fulldf["charge"] = np.where(fulldf.isadc==1,fulldf.data*adcLSB_, (int(tdcOnsetfC_/adcLSB_) + 1.0)*adcLSB_ + fulldf.data*tdcLSB_)
    fulldf["charge_BXm1"] = np.where(fulldf.isadc_BXm1==1,fulldf.data_BXm1*adcLSB_, (int(tdcOnsetfC_/adcLSB_) + 1.0)*adcLSB_ + fulldf.data_BXm1*tdcLSB_)
    
    #print(fulldf)
    #ZS_thr = np.array([1.03 , 1.715, 2.575]) #0.5 MIP threshold, in fC, as found in CMSSW
    ZS_thr = np.array([5, 5, 5]) #5 ADC threshold for each wafer type
    ZS_thr_BXm1 = ZS_thr*5 #2.5 MIP threshold, in fC, as found in CMSSW
    # ZS_thr = np.array([0.7, 0.7, 0.7])
    # ToA_thr = 12. # below this, we don't send ToA, above this we do, 12 fC is threshold listed in TDF

    #drop cells below ZS_thr
    #Correction for leakage from BX1, following Pedro's suggestion
    #https://github.com/cms-sw/cmssw/blob/master/SimCalorimetry/HGCalSimAlgos/interface/HGCalSiNoiseMap.icc#L18-L26
    #80fC for 120um, 160 fC for 200 um and 320 fC for 300 um
    BX1_leakage = np.array([0.066/0.934, 0.153/0.847, 0.0963/0.9037])
    fulldf['data'] = fulldf.data-fulldf.data_BXm1*BX1_leakage[fulldf.gain]
    
    
    df_ZS = fulldf.loc[np.where(fulldf.isadc==0, False, fulldf.data>ZS_thr[fulldf.wafertype])]
    df_ZS['BXM1_readout']= np.where(df_ZS.isadc_BXm1==0, False,  df_ZS.data_BXm1>ZS_thr_BXm1[df_ZS.wafertype])
    df_ZS['TOA_readout'] = (df_ZS.toa>0).astype(int)
    df_ZS['TOT_readout'] = ~df_ZS.isadc
    df_ZS['Bits'] = 16 + 8*(df_ZS.BXM1_readout + df_ZS.TOA_readout)
    
    df_ZS.set_index(['zside','layer','waferu','waferv'],inplace=True)
    df_ZS['HDM'] = df_ZS.wafertype==0
    df_ZS.reset_index(inplace=True)
    df_ZS.set_index(['entry','zside','layer','waferu','waferv'],inplace=True)
    
    #print(df_ZS)
    #print(linkMap)
    df_ZS = df_ZS.reset_index().merge(linkMap,on=['HDM','cellu','cellv']).set_index(['entry','zside','layer','waferu','waferv'])
    #print(df_ZS)
    
    
    calCellDF = df_ZS.reset_index().merge(calibrationCells,on=['HDM','cellu','cellv']).set_index(['entry','zside','layer','waferu','waferv']).fillna(0).drop('isCal',axis=1)
    calCellDF['linkChannel'] = 32
    calCellDF.loc[calCellDF.HDM,'linkChannel'] = 36
    calCellDF = calCellDF.reset_index().set_index(['HDM','eLink','linkChannel'])
    calCellDF.SRAM_read_group = linkMap.set_index(['HDM','eLink','linkChannel']).SRAM_read_group
    calCellDF = calCellDF.reset_index().set_index(['entry','zside','layer','waferu','waferv'])
    #print(calCellDF)
    df_ZS = pd.concat([df_ZS,calCellDF]).sort_index()
    #print(df_ZS)
    
    group = df_ZS.reset_index()[['entry','zside','layer','waferu','waferv','eLink','HDM','Bits','BXM1_readout','TOA_readout']].groupby(['entry','zside','layer','waferu','waferv','eLink'])
    dfBitsElink = group.sum()
    #print(dfBitsElink)
    dfBitsElink['HDM'] = group[['HDM']].any()
    dfBitsElink['occ'] = group['HDM'].count()
    dfBitsElink['eRxPacket_Words'] = (dfBitsElink.Bits/32+1).astype(int) + 2
    print(dfBitsElink)
    
    group = dfBitsElink.reset_index()[['entry','zside','layer','waferu','waferv','HDM','occ','eRxPacket_Words']].groupby(['entry','zside','layer','waferu','waferv'])
    del dfBitsElink
    dfBits = group.sum()
    dfBits['HDM'] = group[['HDM']].any()
    dfBits['NonEmptyLinks'] = group[['HDM']].count()
    dfBits['EmptyLinks'] = np.where(dfBits.HDM,12,6) - dfBits.NonEmptyLinks
    #print(dfBits)
    #print(group.any())
    print(group.count())
    
    evt_headerWords = 2
    evt_trailerWords = 2
    dfBits['TotalWords'] = evt_headerWords + dfBits.eRxPacket_Words + dfBits.EmptyLinks + evt_trailerWords
    dfBits.reset_index(inplace=True)
    dfBits.set_index(['layer','waferu','waferv'],inplace=True)
    
    
   #28443 
    

In [11]:
#----------------------------------------
#Process the tree
#----------------------------------------
_tree = getTree(fName)
Nentries = _tree.num_entries
print(Nentries)
branchesOld = ['hgcdigi_zside','hgcdigi_layer','hgcdigi_waferu','hgcdigi_waferv','hgcdigi_cellu','hgcdigi_cellv','hgcdigi_wafertype','hgcdigi_data','hgcdigi_isadc','hgcdigi_dataBXm1','hgcdigi_isadcBXm1']
branchesNew = ['hgcdigi_zside','hgcdigi_layer','hgcdigi_waferu','hgcdigi_waferv','hgcdigi_cellu','hgcdigi_cellv','hgcdigi_wafertype','hgcdigi_data_BX2','hgcdigi_isadc_BX2','hgcdigi_toa_BX2','hgcdigi_gain_BX2','hgcdigi_data_BX1','hgcdigi_isadc_BX1']

if b'hgcdigi_data' in _tree.keys():
    branches = branchesOld
else:
    branches = branchesNew

outputFile="data_Events_4807336_0.csv"
job=1
layerStart = 5
layerStop=5
chunkSize = 10
maxEvents = 1
isZS = False

N=0
for x in _tree.iterate(branches,entry_stop=maxEvents,step_size=chunkSize):
    print(N)
    N += chunkSize
    layerCut = (x['hgcdigi_layer']>=layerStart) & (x['hgcdigi_layer']<=layerStop)
    df = ak.to_pandas(x[layerCut])
    df.columns = ['zside','layer','waferu','waferv','cellu','cellv','wafertype','data','isadc','toa','gain','data_BXm1','isadc_BXm1']

    #drop subentry from index
    df.reset_index('subentry',drop=True,inplace=True)
    df.reset_index(inplace=True)
    #update entry number for negative endcap
    df['entry'] = df['entry'] + job*jobNumber_eventOffset
    df.loc[df.zside==-1, 'entry'] = df.loc[df.zside==-1, 'entry'] + negZ_eventOffset
    print(df.groupby(['layer','waferu','waferv','cellu','cellv']).count())
    #processDF(df, outputName=outputFile)
    
    
    

File root://cmseos.fnal.gov//store/user/rverma/Output/cms-hgcal-econd/ntuple/ntuple_Events_4807434_0.root
50
0
                                 entry  zside  wafertype  data  isadc  toa  \
layer waferu waferv cellu cellv                                              
5     -11    -7     6     0          1      1          1     1      1    1   
                    7     0          2      2          2     2      2    2   
             -4     9     1          1      1          1     1      1    1   
      -10    -8     7     5          1      1          1     1      1    1   
                    10    9          1      1          1     1      1    1   
                    15    13         1      1          1     1      1    1   
             -7     5     6          1      1          1     1      1    1   
                    6     0          1      1          1     1      1    1   
                          2          1      1          1     1      1    1   
                          6    

In [97]:
linkMap

,HDM,eLink,linkChannel,cellu,cellv,SRAM_read_group
0,False,0,0,0,0,0
1,False,0,1,0,1,0
2,False,0,2,0,2,0
3,False,0,3,0,3,0
4,False,0,4,0,4,0
5,False,0,5,0,5,0
6,False,0,6,0,6,0
7,False,0,7,0,7,0
8,False,0,8,1,0,1
9,False,0,9,1,1,1


In [44]:
sel=(df.layer==5)&(df.waferu==-10)&(df.waferv==-7)&(df.entry==100000)
#tmp_df = df[sel].drop(["isadc", "toa", "gain", "isadc_BXm1", 'charge_BXm1', 'charge', 'layer', 'zside'], axis=1)
tmp_df = df[sel]
newMap = linkMap[(linkMap.cellu==9) & (linkMap.cellv==3)].drop(["SRAM_read_group"], axis=1)
tmp_df2 = tmp_df.reset_index().merge(newMap,on=['cellu', 'cellv'])

In [58]:
df.eLink.unique()

AttributeError: 'DataFrame' object has no attribute 'eLink'

In [52]:
group = tmp_df.reset_index()[['entry','zside','layer','waferu','waferv']].groupby(['entry','zside','layer','waferu','waferv'])
dfBitsElink = group.count()
#dfBitsElink['HDM'] = group[['HDM']].any()
#dfBitsElink['occ'] = group['HDM'].count()
#dfBitsElink['eRxPacket_Words'] = (dfBitsElink.Bits/32+1).astype(int) + 2
dfBitsElink

,,,,
entry,zside,layer,waferu,waferv
100000,1,5,-10,-7


In [43]:
tmp_df.groupby(['cellu', 'cellv']).sum()

entry  waferu  waferv  wafertype       data  data_BXm1
cellu cellv                                                         
6     2      100000     -10      -7          2  11.000000          0
8     3      100000     -10      -7          2  56.000000          0
9     3      100000     -10      -7          2   7.000000          0
      7      100000     -10      -7          2  16.000000          0
10    3      100000     -10      -7          2  28.000000          0
      4      100000     -10      -7          2  49.000000          0
      5      100000     -10      -7          2  22.893438          1
      6      100000     -10      -7          2  11.000000          0
11    5      100000     -10      -7          2  28.786876          2
12    7      100000     -10      -7          2  61.721257         12

In [41]:
tmp_df.groupby(['cellu']).count()

,entry,waferu,waferv,cellv,wafertype,data,data_BXm1
cellu,,,,,,,
6,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1
9,2,2,2,2,2,2,2
10,4,4,4,4,4,4,4
11,1,1,1,1,1,1,1
12,1,1,1,1,1,1,1


In [ ]:
tmp_df.groupby(['cellv']).max()

In [ ]:
tmp_df.groupby(['cellv']).all()

In [ ]:
tmp_df.groupby(['cellv']).any()

In [ ]:
tmp_df.groupby(['cellv']).sum()

In [ ]:
df.groupby(['entry', 'layer', 'waferu', 'waferv']).count()

In [ ]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'loo'],
                    'temp': [1, 2, 3, 5]})
df2 = pd.DataFrame({'lkey': ['foo', 'bar', 'ba', 'foo'],
                    'humidity': [5, 6, 7, 8]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on='lkey')

In [ ]:
pd.merge?

In [ ]:
pd.concat([df1, df2])

In [ ]:
df1.merge(df2, left_on='lkey', right_on='rkey')

In [ ]:
df1 = pd.DataFrame({'city': ['foo', 'bar'], 'b': [1, 2], "b2":[4,5], 'b': [1, 2], "b2":[4,5]})
df2 = pd.DataFrame({'city': ['foo', 'baz'], 'c': [3, 4]})
print(df1)
print(df2)

In [ ]:
df1.merge(df2, on='city')